<a href="https://colab.research.google.com/github/OvaisMemon/LangChain/blob/main/Function_Calling_with_LangChain_RAG_and_Pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project 3: Implement Student Management System using LangChain, Function Calling, RAG and Pinecone DB**

**Step 1: Install Langchain Pinecone**

In [151]:
!pip install -Uq langchain-google-genai langchain-pinecone

**Step 2: Import Necessary Packages and Setup Google Gemini API Key**

In [152]:
import langchain_google_genai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

GoogleAPIKey = userdata.get("GOOGLE_API_KEY")
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash", api_key=GoogleAPIKey)

**Step 3: Import Necessary Packages and Setup Pinecone API Key**

In [153]:
from pinecone import Pinecone, ServerlessSpec

PineconeAPIKey = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=PineconeAPIKey)

**Step 4: Initialize Pinecone Index**

In [154]:
index_name = "langchain-function-calling-with-rag-index-01"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
index = pc.Index(index_name)

**Step 5: Initialize Embedding Model of Google**

In [155]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GoogleAPIKey)

**Step 6: Adding data into Vector Store leveraging Pinecone integration with LangChain**

In [156]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore

student_data = [
    {"studentid": "1", "name": "John Doe", "roll_number": "101"},
    {"studentid": "2", "name": "Jane Smith", "roll_number": "102"},
    {"studentid": "3", "name": "Alice Johnson", "roll_number": "103"},
]

documents = [
    Document(
        page_content=f"Student ID: {record['studentid']}, Name: {record['name']}, Roll Number: {record['roll_number']}",
        metadata={"studentid": record["studentid"], "name": record["name"], "roll_number": record["roll_number"]}
    )
    for record in student_data
]

vector_store = PineconeVectorStore(index=index, embedding=embeddings)
vector_store.add_documents(documents)

# Below code is used to insert data into Pinecone's index without using LangChain
# for student in student_data:
#     vector = embeddings.embed_query(f"{student['name']} {student['roll_number']}")
#     index.upsert([(student["studentid"], vector, student)])

['9f3e7c8a-773c-4b23-9ccd-dca4bd9a9022',
 '0e9ccd8d-7d87-40d5-bb9c-60a4b661c278',
 '672d6f0d-0e2e-4f28-b9ec-f0972e2c7ca6']

**Optional: For testing purpose only**

In [167]:
query1 = "Who is at roll number 101 and 102?"
vector_results = vector_store.similarity_search(query1)

response = model.invoke(f"Answer user queries {query1} from these vector results {vector_results}")
print(response.content)

Roll number 101 is John Doe.  Roll number 102 is Jane Smith.


**Step 7: Define Search function**

In [168]:
from langchain_core.messages import HumanMessage, SystemMessage

def search_student(query: str) -> str:
    """Search for a student in the Pinecone index and return context for LLM."""
    query_vector = embeddings.embed_query(query)
    results = index.query(vector=query_vector, top_k=1, include_metadata=True)
    if results["matches"]:
        student = results["matches"][0]["metadata"]
        context = f"Student Found: ID={student['studentid']}, Name={student['name']}, Roll Number={student['roll_number']}"
        return context
    else:
        return "No matching student found."

**Step 8: Define Tool**

In [169]:
from langchain.tools import Tool

search_tool = Tool(
    name="Search Student",
    func=search_student,
    description="Search for a student by name or roll number and return the context for further response generation."
)

tools = [search_tool]

**Step 9: Initialize LangChain Agent**

In [170]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

**Step 10: Run the Agent**

In [171]:
question = "What is Jane Smith's roll number, return all the information associated with this."
context = agent.run(question)



> Entering new AgentExecutor chain...
Thought: I need to find Jane Smith's information using the Search Student tool.
Action: Search Student
Action Input: Jane Smith
Observation: Student Found: ID=2, Name=Jane Smith, Roll Number=102
Thought:Thought: I now know the final answer.
Final Answer: Jane Smith's roll number is 102.  Her ID is 2.

> Finished chain.


**Step 11: Give the model, the response from the agent and the context to generate the final answer**

In [172]:
response = model.invoke(f"Answer the query {question} using the context {context} in a casual tone.")
print(response.content)

Jane Smith's roll number is 102.  Her ID is 2.  That's all we've got on her!
